# Example: User-interactive app

User selects a machine type, machine, predictor and outcome.  
Output is a scatterplot with the outcome vs. predictor, with a fit line

SM SDK Functions used:
- get_machine_type_names() - to list all machine types, results used in combo box picker
- get_machine_names() - given the machine type above, get all machines of that type, results used in combo box picker
- get_machine_schema() - given the machine from above, get the list of numeric tags, results used in combo box picker
- get_cycles() - get data for the machine/tags selected above, results returned as a pandas data frame and rendered as a table

In [ ]:
from smsdk import client
from datetime import datetime, timedelta
import pandas as pd
import ipywidgets as widgets
import plotly.express as px


In [ ]:
cli = client.Client('my_tenant')
cli.login('apikey', 
          key_id = 'key_from_my_profile', 
          secret_id = 'secret_from_my_profile')

In [ ]:
machine_types = cli.get_machine_type_names()

def dropdown_machine_types_event(change):
    machine_type = change.new
    machines = cli.get_machine_names(source_type=machine_type)
    
    dropdown_machines.options = machines
    dropdown_machines.value = None
    dropdown_machines.disabled = False

    dropdown_outcome.options = []
    dropdown_outcome.value = None
    dropdown_outcome.disabled = True
    
    dropdown_predictor.options = []
    dropdown_predictor.value = None
    dropdown_predictor.disabled = True
    
def dropdown_machines_event(change):
    machine = change.new
    fields = cli.get_machine_schema(machine, types=['int', 'float'])['display'].tolist()
    
    dropdown_outcome.options = fields
    dropdown_outcome.value = None
    dropdown_outcome.disabled = False
    
    dropdown_predictor.options = fields
    dropdown_predictor.value = None
    dropdown_predictor.disabled = False
    
def dropdown_outcome_event(change):
    run_analysis(dropdown_machines.value, change.new, dropdown_predictor.value)

def dropdown_predictor_event(change):
    run_analysis(dropdown_machines.value, dropdown_outcome.value, change.new)
    
def run_analysis(machine, outcome, predictor):
    output_df.clear_output()
    
    if machine and outcome and predictor:
    
        cols = ['Machine', 'End Time', outcome, predictor]

        query = {'Machine': machine,
                 'End Time__gt': datetime.utcnow() - timedelta(7),
                 '_limit': 500, 
                 '_only': cols
        }
        
        df = cli.get_cycles(**query)
        
        with output_df:
            if len(df):
                df.set_index('End Time', inplace=True)
                
                fig = px.scatter(df,
                         x = predictor,
                         y = outcome,
                         trendline = 'ols')
                
                display(fig)
                display(df[[outcome, predictor]])
            else:
                display(pd.DataFrame())

In [ ]:
dropdown_machine_types = widgets.Dropdown(options = machine_types,
                                          value = None,
                                          description = 'Type',
                                          disabled = False)
dropdown_machines = widgets.Dropdown(options = [],
                                     value = None,
                                     description = 'Machine',
                                     disabled = True)
dropdown_outcome = widgets.Dropdown(options = [],
                                    value = None,
                                    description = 'Outcome',
                                    disabled = True)
dropdown_predictor = widgets.Dropdown(options = [],
                                      value = None,
                                      description = 'Predictor',
                                      disabled = True)
output_df = widgets.Output()

dropdown_machine_types.observe(dropdown_machine_types_event, names='value')
dropdown_machines.observe(dropdown_machines_event, names='value')
dropdown_outcome.observe(dropdown_outcome_event, names='value')
dropdown_predictor.observe(dropdown_predictor_event, names='value')

display(dropdown_machine_types)
display(dropdown_machines)
display(dropdown_outcome)
display(dropdown_predictor)
display(output_df)